In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

file_path = '/content/drive/My Drive/Sports Modeling/mlb/mlb_proccessed_v1.csv'
df = pd.read_csv(file_path)

<ipython-input-187-63933633908f>:4: DtypeWarning: Columns (4,321,330,335,463,527,594,598,600,602,606,608,717,718,719,720,721,722,723,724) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
print(df.columns.tolist())


['id', 'status', 'scheduled', 'attendance', 'split_squad', 'final_inning', 'final_inning_half', 'home_name', 'home_hits', 'home_errors', 'home_win', 'home_loss', 'home_scoring', 'home_hitting_overall_ab', 'home_hitting_overall_lob', 'home_hitting_overall_rbi', 'home_hitting_overall_abhr', 'home_hitting_overall_abk', 'home_hitting_overall_bip', 'home_hitting_overall_babip', 'home_hitting_overall_bbk', 'home_hitting_overall_bbpa', 'home_hitting_overall_iso', 'home_hitting_overall_obp', 'home_hitting_overall_ops', 'home_hitting_overall_seca', 'home_hitting_overall_slg', 'home_hitting_overall_xbh', 'home_hitting_overall_pitch_count', 'home_hitting_overall_lob_risp_2out', 'home_hitting_overall_team_lob', 'home_hitting_overall_ab_risp', 'home_hitting_overall_hit_risp', 'home_hitting_overall_rbi_2out', 'home_hitting_overall_linedrive', 'home_hitting_overall_groundball', 'home_hitting_overall_popup', 'home_hitting_overall_flyball', 'home_hitting_overall_ap', 'home_hitting_overall_avg', 'home_h

In [ ]:
import pandas as pd

# Get the count of null values in each column
null_counts = df.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

# Optionally, save the null counts to a CSV file for further analysis
null_counts.to_csv('null_value_counts.csv', header=['null_counts'])

# If you want to download the file, use the following code
from google.colab import files
files.download('null_value_counts.csv')

id                 4
status             4
scheduled          4
attendance      2870
split_squad    18420
               ...  
day                4
time               4
home_score         4
away_score         4
total              4
Length: 732, dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Droping columns due to too many null values!

In [ ]:
# Generate a list of columns with more than 4000 null values
columns_with_many_nulls = df.columns[df.isnull().sum() > 4000].tolist()

# Drop the identified columns
df = df.drop(columns=columns_with_many_nulls)

In [ ]:
#Dropping more columns due to irrelevance
columns_irrelevance = ['id', 'status','home_scoring','away_scoring', 'officials', 'duration','pitching_win_status', 'pitching_loss_status', 'home_pitching_starters_era',
 'away_pitching_starters_era', 'away_pitching_bullpen_era','home_pitching_bullpen_era','time', 'scheduled' ]

# Drop the identified columns
df = df.drop(columns=columns_irrelevance)

In [ ]:
# Identify columns with null values
columns_with_nulls = df.columns[df.isnull().any()].tolist()

# Identify numeric columns with null values
numeric_columns_with_nulls = df.select_dtypes(include=['number']).columns[df.select_dtypes(include=['number']).isnull().any()].tolist()

# Step 1: Group-Based Mean Imputation (based on home_name and away_name) for numeric columns
for column in numeric_columns_with_nulls:
    df[column] = df.groupby(['home_name', 'away_name'])[column].transform(lambda x: x.fillna(x.mean()))

# Step 2: Overall Median Imputation for any remaining nulls in numeric columns
for column in numeric_columns_with_nulls:
    df[column].fillna(df[column].median(), inplace=True)

# Step 3: Overall Mode Imputation for any remaining nulls in non-numeric columns
non_numeric_columns_with_nulls = [col for col in columns_with_nulls if col not in numeric_columns_with_nulls]
for column in non_numeric_columns_with_nulls:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [ ]:
# Now check for non-numerical columns
non_numerical_data = df.select_dtypes(exclude=['int64', 'float64'])
print(non_numerical_data.head())

  final_inning_half     home_name  away_name
0                 B        Angels    Dodgers
1                 B          Reds   Mariners
2                 T      Mariners     Royals
3                 B  Diamondbacks    Brewers
4                 B       Pirates  Blue Jays


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to each categorical column
df['final_inning_half_encoded'] = label_encoder.fit_transform(df['final_inning_half'])
df['home_name_encoded'] = label_encoder.fit_transform(df['home_name'])
df['away_name_encoded'] = label_encoder.fit_transform(df['away_name'])

# Drop the original columns if you no longer need them
df = df.drop(['final_inning_half', 'home_name', 'away_name'], axis=1)

<ipython-input-194-a29ecbd62e87>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['final_inning_half_encoded'] = label_encoder.fit_transform(df['final_inning_half'])
<ipython-input-194-a29ecbd62e87>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['home_name_encoded'] = label_encoder.fit_transform(df['home_name'])
<ipython-input-194-a29ecbd62e87>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# Define the model
model = RandomForestClassifier(random_state=42)

# Define X by dropping the 'result' column
X = df.drop(columns=['result'])

# Define y as the 'result' column
y = df['result']

# Define RFE and select the top 10 features
rfe = RFE(estimator=model, n_features_to_select=50, step=1)
rfe = rfe.fit(X, y)

# Get the selected features
selected_features = X.columns[rfe.support_]
print("Selected Features:", selected_features)

# Optionally, check the ranking of all features
print("Feature Ranking:", rfe.ranking_)

Selected Features: Index(['attendance', 'final_inning', 'home_hits', 'home_win', 'home_loss',
       'home_hitting_overall_pitch_count', 'home_hitting_overall_runs_total',
       'home_pitching_overall_pitch_count', 'home_pitching_overall_ip_1',
       'home_pitching_overall_ip_2', 'home_pitching_overall_runs_total',
       'home_pitching_overall_runs_earned', 'home_pitching_overall_games_win',
       'home_pitching_overall_games_loss', 'home_pitching_overall_games_save',
       'home_fielding_overall_po', 'away_hits',
       'away_hitting_overall_pitch_count', 'away_hitting_overall_runs_total',
       'away_pitching_overall_era', 'away_pitching_overall_ip_1',
       'away_pitching_overall_ip_2', 'away_pitching_overall_games_win',
       'away_pitching_overall_games_loss', 'away_pitching_overall_games_save',
       'away_pitching_overall_games_blown_save', 'away_fielding_overall_po',
       'away_win', 'away_loss', 'pitching_win_save', 'pitching_win_blown_save',
       'pitching_loss_s

In [ ]:
feature_rankings = [
  1, 1, 1, 22, 1, 1, 42, 408, 37, 327, 235, 210, 234, 271, 226, 170, 53, 39,
  141, 47, 437, 1, 325, 281, 192, 175, 346, 361, 220, 383, 294, 12, 59, 278, 371, 300,
  399, 400, 66, 353, 338, 324, 412, 391, 136, 480, 1, 173, 174, 89, 86, 409, 345, 79,
  396, 375, 459, 425, 450, 262, 389, 422, 439, 308, 417, 448, 401, 426, 416, 397, 109, 225,
  20, 301, 87, 1, 132, 75, 1, 1, 128, 138, 168, 80, 54, 106, 45, 71, 152, 378,
  167, 285, 77, 295, 270, 238, 115, 35, 297, 1, 85, 1, 40, 216, 114, 97, 367, 163,
  7, 273, 150, 410, 290, 368, 194, 364, 135, 162, 187, 38, 329, 286, 296, 119, 49, 78,
  462, 1, 1, 1, 29, 24, 1, 44, 222, 465, 147, 2, 90, 127, 185, 261, 292, 431,
  282, 69, 221, 444, 291, 1, 31, 43, 218, 8, 269, 252, 149, 289, 358, 263, 116, 143,
  9, 81, 62, 178, 1, 352, 214, 145, 159, 330, 373, 332, 369, 236, 19, 169, 275, 413,
  438, 317, 354, 142, 386, 442, 355, 407, 387, 129, 468, 1, 104, 130, 125, 94, 447, 418,
  63, 393, 247, 461, 420, 423, 246, 374, 380, 360, 310, 276, 455, 176, 342, 436, 370, 93,
  182, 1, 337, 15, 17, 328, 264, 1, 1, 84, 60, 146, 72, 103, 95, 67, 102, 313,
  319, 154, 256, 101, 402, 229, 250, 166, 209, 197, 5, 41, 28, 120, 46, 153, 52, 414,
  305, 14, 333, 260, 427, 356, 382, 204, 343, 244, 404, 224, 379, 350, 233, 268, 205, 3,
  164, 453, 1, 1, 1, 4, 1, 1, 73, 131, 464, 253, 32, 161, 248, 172, 349, 193,
  446, 249, 113, 180, 428, 255, 1, 1, 1, 11, 1, 1, 1, 6, 155, 245, 137, 139,
  25, 206, 419, 92, 334, 98, 309, 232, 357, 390, 315, 199, 196, 432, 304, 339, 362, 321,
  254, 259, 318, 151, 117, 82, 21, 481, 288, 55, 280, 326, 463, 242, 451, 186, 351, 366,
  231, 340, 284, 443, 376, 314, 433, 471, 190, 460, 1, 1, 475, 472, 473, 156, 331, 388,
  74, 335, 293, 445, 88, 287, 177, 183, 392, 277, 398, 312, 134, 211, 405, 336, 320, 243,
  213, 36, 227, 100, 181, 316, 240, 124, 467, 307, 56, 298, 165, 458, 274, 424, 217, 201,
  148, 184, 200, 96, 415, 157, 341, 440, 61, 469, 477, 1, 1, 33, 64, 70, 223, 283,
  122, 202, 1, 257, 435, 34, 107, 30, 158, 430, 347, 394, 123, 212, 265, 434, 239, 441,
  381, 365, 219, 406, 237, 57, 195, 65, 16, 474, 344, 51, 160, 348, 454, 228, 421, 133,
  266, 76, 179, 359, 403, 363, 372, 83, 429, 482, 251, 457, 1, 1, 483, 470, 478, 110,
  91, 198, 48, 302, 279, 452, 111, 27, 140, 126, 322, 189, 411, 215, 118, 241, 311, 323,
  105, 377, 207, 10, 121, 144, 68, 208, 203, 99, 476, 112, 171, 230, 303, 456, 272, 385,
  108, 188, 258, 267, 191, 306, 395, 384, 299, 449, 23, 466, 479, 1, 1, 1, 58, 13,
  1, 1, 1, 1, 1, 1, 18, 1, 26, 50
]

# Define X by dropping the 'result' column
X = df.drop(columns=['result'])

# Create a DataFrame with features and their rankings
feature_rankings = pd.DataFrame({
    'Feature': X.columns,  # The feature names
    'Ranking': feature_rankings  # The rankings from RFE
})

# Sort the features by their ranking
feature_rankings = feature_rankings.sort_values(by='Ranking')

# Select features with a ranking less than or equal to 5
important_features = feature_rankings[feature_rankings['Ranking'] <= 5]['Feature']
important_features_list = important_features.tolist()

print("Selected Important Features with Ranking <= 5:\n", important_features_list)

Selected Important Features with Ranking <= 5:
 ['attendance', 'home_pitching_bullpen_games_win', 'home_pitching_starters_games_loss', 'home_pitching_starters_games_win', 'pitching_loss_hold', 'pitching_loss_save', 'pitching_win_blown_save', 'home_hitting_overall_runs_total', 'home_pitching_overall_runs_total', 'away_loss', 'away_win', 'away_fielding_overall_po', 'away_pitching_overall_games_blown_save', 'away_pitching_overall_games_save', 'away_pitching_overall_games_loss', 'away_pitching_overall_games_win', 'away_pitching_overall_ip_2', 'away_pitching_overall_ip_1', 'away_pitching_overall_era', 'away_hitting_overall_runs_total', 'away_hitting_overall_pitch_count', 'home_pitching_overall_pitch_count', 'home_pitching_overall_ip_1', 'home_pitching_overall_ip_2', 'away_hits', 'home_fielding_overall_po', 'home_pitching_overall_games_save', 'home_pitching_overall_games_loss', 'home_pitching_overall_games_win', 'home_pitching_overall_runs_earned', 'home_pitching_bullpen_games_loss', 'away_p

In [ ]:
important_features_list =  ['attendance', 'home_pitching_bullpen_games_win', 'home_pitching_starters_games_loss', 'home_pitching_starters_games_win',
                            'pitching_loss_hold', 'pitching_loss_save', 'pitching_win_blown_save', 'home_hitting_overall_runs_total', 'home_pitching_overall_runs_total',
                            'away_loss', 'away_win', 'away_fielding_overall_po', 'away_pitching_overall_games_blown_save', 'away_pitching_overall_games_save',
                            'away_pitching_overall_games_loss', 'away_pitching_overall_games_win', 'away_pitching_overall_ip_2', 'away_pitching_overall_ip_1',
                            'away_pitching_overall_era', 'away_hitting_overall_runs_total', 'away_hitting_overall_pitch_count', 'home_pitching_overall_pitch_count',
                            'home_pitching_overall_ip_1', 'home_pitching_overall_ip_2', 'away_hits', 'home_fielding_overall_po', 'home_pitching_overall_games_save',
                            'home_pitching_overall_games_loss', 'home_pitching_overall_games_win', 'home_pitching_overall_runs_earned', 'home_pitching_bullpen_games_loss',
                            'away_pitching_starters_pitch_count', 'pitching_win_save', 'month', 'away_pitching_bullpen_games_loss', 'home_hitting_overall_pitch_count',
                            'away_pitching_bullpen_games_win', 'away_pitching_bullpen_games_save', 'home_score', 'final_inning_half_encoded', 'total', 'day', 'home_loss',
                            'away_pitching_starters_games_loss', 'away_pitching_starters_games_win', 'home_win', 'home_hits', 'year', 'final_inning', 'away_score',
                            'home_fielding_overall_tc', 'away_pitching_overall_games_svo', 'away_pitching_overall_games_hold', 'away_pitching_overall_runs_total']

In [ ]:
# Optionally, check if there are any NaNs introduced by the rolling or division operations
# Get the count of null values in each column
null_counts = df.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

home_win_ratio                 203
away_win_ratio                 217
home_pitching_effectiveness    233
away_pitching_effectiveness    233
home_pitch_count_efficiency    746
away_pitch_count_efficiency    746
home_runs_allowed_per_game       4
away_runs_allowed_per_game       1
home_win_consistency             4
away_win_consistency             4
home_loss_consistency            4
away_loss_consistency            4
dtype: int64


In [ ]:
# Drop rows with any null values
df = df.dropna()

In [ ]:
pregame_features = [ 'year', 'month', 'day', 'home_name_encoded', 'away_name_encoded']

all_features = ['attendance', 'home_pitching_bullpen_games_win', 'home_pitching_starters_games_loss', 'home_pitching_starters_games_win',
                'pitching_loss_hold', 'pitching_loss_save', 'pitching_win_blown_save', 'home_hitting_overall_runs_total', 'home_pitching_overall_runs_total',
                'away_loss', 'away_win', 'away_fielding_overall_po', 'away_pitching_overall_games_blown_save', 'away_pitching_overall_games_save',
                'away_pitching_overall_games_loss', 'away_pitching_overall_games_win', 'away_pitching_overall_ip_2', 'away_pitching_overall_ip_1',
                'away_pitching_overall_era', 'away_hitting_overall_runs_total', 'away_hitting_overall_pitch_count', 'home_pitching_overall_pitch_count',
                'home_pitching_overall_ip_1', 'home_pitching_overall_ip_2', 'away_hits', 'home_fielding_overall_po', 'home_pitching_overall_games_save',
                'home_pitching_overall_games_loss', 'home_pitching_overall_games_win', 'home_pitching_overall_runs_earned', 'home_pitching_bullpen_games_loss',
                'away_pitching_starters_pitch_count', 'pitching_win_save', 'month', 'away_pitching_bullpen_games_loss', 'home_hitting_overall_pitch_count',
                'away_pitching_bullpen_games_win', 'away_pitching_bullpen_games_save', 'home_score', 'final_inning_half_encoded', 'total', 'day', 'home_loss',
                'away_pitching_starters_games_loss', 'away_pitching_starters_games_win', 'home_win', 'home_hits', 'year', 'final_inning', 'away_score',
                'home_fielding_overall_tc', 'away_pitching_overall_games_svo', 'away_pitching_overall_games_hold', 'away_pitching_overall_runs_total',
                'home_name_encoded', 'away_name_encoded', 'result']


In [ ]:
# List of columns that are not numerical
non_numerical_columns = df[pregame_features].select_dtypes(exclude=['number']).columns.tolist()

# Display the non-numerical columns
print(non_numerical_columns)

[]


In [ ]:
# List of columns that are not numerical
non_numerical_columns = df[all_features].select_dtypes(exclude=['number']).columns.tolist()

# Display the non-numerical columns
print(non_numerical_columns)

[]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# creatiing new dataframe with all new features
df_new = df[all_features]

# Sort the dataset by 'season_year', 'month', and 'day'
df_new_sorted = df_new.sort_values(by=['year', 'month', 'day'], ascending=[True, True, True])

# Split the data into training and validation sets (80% for training, 20% for validation)
train_val_data, test_data = train_test_split(df_new_sorted, test_size=0.01, shuffle=False)

# Further split the training and validation set into 80% training and 20% validation
train_data, val_data = train_test_split(train_val_data, test_size=0.3, shuffle=False)

# Step 4: Create the test dataset with only the before_game_features
Before_game_features = [ 'year', 'month', 'day', 'home_name_encoded', 'away_name_encoded','result']

test_data = test_data[Before_game_features]

# Displaying the dataset shapes to the user for verification
print("Training Data Shape:", train_data.shape)
print("Validation Data Shape:", val_data.shape)
print("Testing Data Shape:", test_data.shape)

Training Data Shape: (14135, 57)
Validation Data Shape: (6058, 57)
Testing Data Shape: (204, 6)


In [ ]:
import numpy as np
# X is the DataFrame containing all features
X = df[all_features]

# Check for infinite values in X
infinite_mask = np.isinf(X)

# Get the count of infinite values for each column
inf_counts = infinite_mask.sum(axis=0)

# Filter out columns with infinite values and get the counts
inf_columns_with_counts = inf_counts[inf_counts > 0]

# Print the results
print("Columns with infinite values and their counts in X:")
print(inf_columns_with_counts)

Columns with infinite values and their counts in X:
Series([], dtype: int64)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.utils import class_weight

# Features and target for training and validation
X_train = train_data[all_features].drop(columns=['result'])  # Drop the target column from features
y_train = to_categorical(train_data['result'] + 1, num_classes=3)  # Shift to make classes 0, 1, 2

X_val = val_data[all_features].drop(columns=['result'])  # Drop the target column from features
y_val = to_categorical(val_data['result'] + 1, num_classes=3)  # Shift to make classes 0, 1, 2

# Check for infinite values and replace them with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val.replace([np.inf, -np.inf], np.nan, inplace=True)

# Impute NaN values (mean imputation is one approach)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)

# Normalize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Create the model
model = Sequential()
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))  # Add dropout to reduce overfitting
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Compute class weights for imbalanced datasets
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# Train the model using the training and validation sets
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weights)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy:.2f}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5990 - loss: 2.1258 - val_accuracy: 0.9946 - val_loss: 0.8208
Epoch 2/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8936 - loss: 1.0666 - val_accuracy: 0.9990 - val_loss: 0.5322
Epoch 3/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9283 - loss: 0.7254 - val_accuracy: 0.9985 - val_loss: 0.3792
Epoch 4/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9386 - loss: 0.5287 - val_accuracy: 1.0000 - val_loss: 0.2792
Epoch 5/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9410 - loss: 0.4251 - val_accuracy: 1.0000 - val_loss: 0.2175
Epoch 6/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9477 - loss: 0.3505 - val_accuracy: 1.0000 - val_loss: 0.1729
Epoch 7/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9416 - loss: 0.3407 - val_accuracy: 1.0000 - val_loss: 0.1549
Epoch 8/10
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9474 - loss: 0.2732 - val_accuracy: 1.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions on the test set
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert one-hot encoded predictions back to class labels
y_test_classes = np.argmax(y_val, axis=1)  # Convert one-hot encoded test labels back to class labels

# Confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix:
[[2841    0    0]
 [   0   11    0]
 [   0    0 3206]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2841
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00      3206

    accuracy                           1.00      6058
   macro avg       1.00      1.00      1.00      6058
weighted avg       1.00      1.00      1.00      6058



In [ ]:
X_train = train_data[all_features]
X_train.to_csv('Train_data.csv', index=False)

### Recreating model using Focal Loss


> To address class imbalance by down-weighting easy examples and focusing training on harder

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical

# Features and target for training and validation
X_train = train_data[all_features].drop(columns=['result'])  # Drop the target column from features
y_train = to_categorical(train_data['result'] + 1, num_classes=3)  # Shift to make classes 0, 1, 2

X_val = val_data[all_features].drop(columns=['result'])  # Drop the target column from features
y_val = to_categorical(val_data['result'] + 1, num_classes=3)  # Shift to make classes 0, 1, 2

# Check for infinite values and replace them with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val.replace([np.inf, -np.inf], np.nan, inplace=True)

# Impute NaN values (mean imputation)
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_val = imputer.transform(X_val)

# Normalize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define focal loss function
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        eps = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, eps, 1. - eps)
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (tf.keras.backend.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (tf.keras.backend.ones_like(y_true) - y_true) * (tf.keras.backend.ones_like(y_true) - y_pred)
        fl = - alpha_t * tf.keras.backend.pow((tf.keras.backend.ones_like(y_true) - p_t), gamma) * tf.keras.backend.log(p_t)
        return tf.keras.backend.mean(fl)
    return focal_loss_fixed

# Build the model to avoid overfitting, adding dropout layers and regularization
model = Sequential()
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))  # Adjusted dropout rate
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

# Compile the model with focal loss
model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=.25), metrics=['accuracy'])

# Calculate class weights for imbalanced dataset
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # Increased patience
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weights)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy:.2f}')

# Generate confusion matrix and classification report
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_val, axis=1)

print("Confusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


442/442 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7757 - loss: 0.6731 - val_accuracy: 0.9982 - val_loss: 0.0297
Epoch 2/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9651 - loss: 0.0320 - val_accuracy: 0.9982 - val_loss: 0.0144
Epoch 3/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9606 - loss: 0.0243 - val_accuracy: 0.9982 - val_loss: 0.0119
Epoch 4/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9598 - loss: 0.0224 - val_accuracy: 0.9982 - val_loss: 0.0109
Epoch 5/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9618 - loss: 0.0217 - val_accuracy: 0.9982 - val_loss: 0.0107
Epoch 6/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9628 - loss: 0.0198 - val_accuracy: 0.9982 - val_loss: 0.0104
Epoch 7/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9642 - loss: 0.0191 - val_accuracy: 0.9982 - val_loss: 0.0102
Epoch 8/20
442/442 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9664 - loss: 0.0191 - val_accuracy: 0.9982 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Function to Retrieve Historical Data

In [ ]:
import numpy as np

# List of non-float columns that should be converted to nearest integer
non_float_columns = ['away_hits', 'home_score', 'final_inning_half_encoded', 'total', 'home_hits', 'final_inning', 'away_score']

def retrieve_historical_data(home_team, away_team, game_date, historical_df, all_features_new, lookback_games=5):
    # Unpack the game_date tuple
    game_year, game_month, game_day = game_date

    # Filter the dataframe to include only important features plus date and team identifiers
    important_features = [f for f in all_features_new if f not in Before_game_features]
    relevant_columns = important_features + ['home_name_encoded', 'away_name_encoded', 'year', 'month', 'day']

    # Select only relevant columns
    historical_df = historical_df[relevant_columns]

    # Drop rows with NaN values
    historical_df.dropna(inplace=True)

    # First, sort the data by season year, month, and day in descending order
    historical_df = historical_df.sort_values(by=['year', 'month', 'day'], ascending=False)

    # Now filter for home-away combinations (either home vs away or away vs home)
    past_games_combined = historical_df[
        (
            ((historical_df['home_name_encoded'] == home_team) & (historical_df['away_name_encoded'] == away_team)) |
            ((historical_df['home_name_encoded'] == away_team) & (historical_df['away_name_encoded'] == home_team))
        )
    ]

    # Further filter to ensure that the games occurred before the specific game date
    past_games_combined = past_games_combined[
        (
            (past_games_combined['year'] < game_year) |  # All games from previous years
            (
                (past_games_combined['year'] == game_year) &  # Same year, but before the specific game date
                (
                    (past_games_combined['month'] < game_month) |
                    ((past_games_combined['month'] == game_month) & (past_games_combined['day'] < game_day))
                )
            )
        )
    ]

    # Select only the most recent `lookback_games` number of games
    past_games_combined = past_games_combined.head(lookback_games)

    # Calculate historical features using the mean for each of the important features
    historical_features = {}

    for feature in important_features:
        if feature in past_games_combined.columns:
            feature_mean = past_games_combined[feature].mean()
            # If the feature is in the non-float columns, round to nearest integer
            if feature in non_float_columns:
                feature_mean = np.round(feature_mean, 0)  # Round to nearest integer
            historical_features[feature] = feature_mean

    return historical_features

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

main_df_columns = [
                'attendance', 'home_pitching_bullpen_games_win', 'home_pitching_starters_games_loss', 'home_pitching_starters_games_win',
                'pitching_loss_hold', 'pitching_loss_save', 'pitching_win_blown_save', 'home_hitting_overall_runs_total', 'home_pitching_overall_runs_total',
                'away_loss', 'away_win', 'away_fielding_overall_po', 'away_pitching_overall_games_blown_save', 'away_pitching_overall_games_save',
                'away_pitching_overall_games_loss', 'away_pitching_overall_games_win', 'away_pitching_overall_ip_2', 'away_pitching_overall_ip_1',
                'away_pitching_overall_era', 'away_hitting_overall_runs_total', 'away_hitting_overall_pitch_count', 'home_pitching_overall_pitch_count',
                'home_pitching_overall_ip_1', 'home_pitching_overall_ip_2', 'away_hits', 'home_fielding_overall_po', 'home_pitching_overall_games_save',
                'home_pitching_overall_games_loss', 'home_pitching_overall_games_win', 'home_pitching_overall_runs_earned', 'home_pitching_bullpen_games_loss',
                'away_pitching_starters_pitch_count', 'pitching_win_save', 'month', 'away_pitching_bullpen_games_loss', 'home_hitting_overall_pitch_count',
                'away_pitching_bullpen_games_win', 'away_pitching_bullpen_games_save', 'home_score', 'final_inning_half_encoded', 'total', 'day', 'home_loss',
                'away_pitching_starters_games_loss', 'away_pitching_starters_games_win', 'home_win', 'home_hits', 'year', 'final_inning', 'away_score',
                'home_fielding_overall_tc', 'away_pitching_overall_games_svo', 'away_pitching_overall_games_hold', 'away_pitching_overall_runs_total',
                'home_name_encoded', 'away_name_encoded']

def create_feature_vector(home_team, away_team, before_game_features, game_date, historical_df, all_features_new, scaler):
    # Ensure historical_df is not modified in place (avoid SettingWithCopyWarning)
    historical_df = historical_df.copy()

    # Drop missing values (if required)
    historical_df.dropna(inplace=True)

    # Combine before-game features with historical features
    historical_features = retrieve_historical_data(home_team, away_team, game_date, historical_df, all_features_new)

    # Ensure both historical_features and before_game_features are dictionaries
    if not isinstance(historical_features, dict):
        raise TypeError("Expected historical_features to be a dictionary.")

    if not isinstance(before_game_features, dict):
        raise TypeError("Expected before_game_features to be a dictionary.")

    feature_vector = {**historical_features, **before_game_features}

    # Convert to DataFrame for model input
    feature_vector_df = pd.DataFrame([feature_vector])

    # Reorder the columns to match the required order (main_df_columns)
    feature_vector_df = feature_vector_df.reindex(columns=main_df_columns)

    # Ensure no missing columns in the feature vector
    missing_cols = set(main_df_columns) - set(feature_vector_df.columns)
    if missing_cols:
        raise ValueError(f"Missing columns in feature vector: {missing_cols}")

    # Scale the features using the provided scaler
    feature_vector_scaled = pd.DataFrame(scaler.transform(feature_vector_df), columns=feature_vector_df.columns)

    # Return the scaled feature vector
    return feature_vector_scaled

In [ ]:
def predict_game_outcome(home_team, away_team, before_game_features, game_date, historical_df, important_features, model, scaler):
    # Create the feature vector
    feature_vector_df = create_feature_vector(home_team, away_team, before_game_features, game_date, historical_df, important_features, scaler)

    # Make the prediction
    prediction = model.predict(feature_vector_df)

    # Get the index of the highest probability class
    predicted_class = np.argmax(prediction, axis=1)[0]  # Extract the class index (0, 1, or 2)

    # Map the predicted class index to the corresponding label
    prediction_label = "Home Win" if predicted_class == 2 else "Away Win" if predicted_class == 0 else "Draw"
    return prediction_label

In [ ]:

import pandas as pd
from datetime import datetime

# Assuming historical_df, model, scaler, and important_features are already defined and available
pregame_features = [ 'year', 'month', 'day', 'home_name_encoded', 'away_name_encoded']

# List to store the predictions
predictions = []

# Iterate over each row in X_test
for index, row in test_data.iterrows():
    # Extract the relevant features for the prediction
    before_game_features = {
        'year': row['year'],
        'home_name_encoded': row['home_name_encoded'],
        'away_name_encoded': row['away_name_encoded'],
        'month': row['month'],
        'day': row['day']
    }

    # Construct the game_date from year, month, and day
    game_date = (row['year'], row['month'], row['day'])

    # Predict the game outcome using the extracted features
    prediction_label = predict_game_outcome(
        home_team = row['home_name_encoded'],
        away_team = row['away_name_encoded'],
        before_game_features = before_game_features,
        game_date = game_date,
        historical_df = df,  # This should be your historical data DataFrame
        important_features = all_features,  # Pass the list of important features here
        model = model,
        scaler=scaler,  # Assuming you have a scaler, pass None if no scaling is needed
    )

    # Append the prediction to the list
    predictions.append(prediction_label)

# Add the predictions to X_test DataFrame for comparison
test_data['predicted_outcome'] = predictions

# Optionally, print the DataFrame to see the predictions
print(test_data[['home_name_encoded', 'away_name_encoded', 'predicted_outcome', 'result']])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


<ipython-input-211-e40cc7429a5f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_df.dropna(inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


<ipython-input-211-e40cc7429a5f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_df.dropna(inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


...

<ipython-input-211-e40cc7429a5f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_df.dropna(inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


<ipython-input-211-e40cc7429a5f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_df.dropna(inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
       home_name_encoded  away_name_encoded predicted_outcome  result
19197                 14                 40          Home Win       1
19587                 18                 15          Home Win      -1
19678                  1                 50          Away Win      -1
19806                 41                 52          Away Win      -1
19834                 20                 44          Home Win       1
...                  ...                ...               ...     ...
21158                 18                 40          Home Win       1
21183                 33                 52          Away Win       1
21236                 13                 15          Away Win       1
21276                 41                 36          Away Win      -1
20027                 20                 27          Home Win       1

[204 rows x 4 columns]


<ipython-input-211-e40cc7429a5f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_df.dropna(inplace=True)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score

# Step 1: Adjust the 'result' column to match the labels
# Convert -1 to 0, 0 stays the same, and 1 to 2
test_data['result'] = test_data['result'].map({-1: 0, 0: 1, 1: 2})

# Step 2: Print the DataFrame to verify the conversion and predictions
print(test_data[['home_name_encoded', 'away_name_encoded', 'predicted_outcome', 'result']])


       home_name_encoded  away_name_encoded predicted_outcome  result
19197                 14                 40          Home Win       2
19587                 18                 15          Home Win       0
19678                  1                 50          Away Win       0
19806                 41                 52          Away Win       0
19834                 20                 44          Home Win       2
...                  ...                ...               ...     ...
21158                 18                 40          Home Win       2
21183                 33                 52          Away Win       2
21236                 13                 15          Away Win       2
21276                 41                 36          Away Win       0
20027                 20                 27          Home Win       2

[204 rows x 4 columns]


In [ ]:

# Step 3: Calculate the accuracy score
# Convert 'predicted_outcome' from labels ("Home Win", "Away Win", "Draw") to numerical values (2, 0, 1)
test_data['predicted_outcome_num'] = test_data['predicted_outcome'].map({"Home Win": 2, "Away Win": 0, "Draw": 1})

# Calculate accuracy
accuracy = accuracy_score(test_data['result'], test_data['predicted_outcome_num'])

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.53


In [ ]:
print(test_data['predicted_outcome_num'].value_counts())

predicted_outcome_num
0    110
2     94
Name: count, dtype: int64
